In [ ]:
import requests
import json
from dotenv import load_dotenv
import os
from openai import OpenAI

# --- Simap API Konfiguration ---
SIMAP_BASE_URL = "https://simap.ch" # WICHTIG: Korrekte Basis-URL hier eintragen!
SIMAP_SEARCH_ENDPOINT = "/api/publications/v2/project/project-search"
SIMAP_DETAIL_ENDPOINT_TEMPLATE = "/api/publications/v1/project/{projectId}/publication-details/{publicationId}"


load_dotenv(override=True)

webhook_url = os.environ["SLACK_WEBHOOK_BIDBOT"]

client = OpenAI(
    # This is the default and can be omitted
    api_key= os.environ["OPENAI_API_KEY"]
)



In [39]:
def call_simap_api(endpoint_url, params=None):
    """Ruft einen Simap API Endpunkt auf und gibt die JSON-Antwort zurück."""
    try:
        print(f"INFO: Rufe Simap API auf: {endpoint_url} mit Parametern: {params}")
        # Da Sie sagten, keine Authentifizierung nötig:
        response = requests.get(endpoint_url, params=params)
        response.raise_for_status()  # Löst einen Fehler aus bei HTTP-Fehlercodes 4xx/5xx
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"FEHLER: HTTP-Fehler beim Aufruf der Simap API ({endpoint_url}): {http_err}")
        print(f"FEHLER: Antwort-Text: {response.text if response else 'Keine Antwort'}")
    except requests.exceptions.RequestException as e:
        print(f"FEHLER: Allgemeiner Fehler beim Aufruf der Simap API ({endpoint_url}): {e}")
    except json.JSONDecodeError as e:
        print(f"FEHLER: Antwort der Simap API ({endpoint_url}) ist kein valides JSON: {e}")
        print(f"FEHLER: Antwort-Text: {response.text if response else 'Keine Antwort'}")
    return None



In [64]:
print("--- Schritt 1: Alle relevanten Projektseiten abrufen (mit Paginierung) ---")
search_url = SIMAP_BASE_URL + SIMAP_SEARCH_ENDPOINT

# Ihre gewünschten initialen Query-Parameter
# WICHTIG: Prüfen Sie in der Simap Swagger-UI, ob es einen Parameter gibt,
# um direkt nach pubType zu filtern, z.B. pubTypes="tender,advance_notice"
# Wenn ja, fügen Sie ihn hier hinzu!
initial_query_params = {
    "lang": "de",
    "processTypes": "open",
    "cpvCodes": ["72000000","48000000"], 
    "newestPublicationFrom": "2025-05-27"
    # "pubTypes": "tender,advance_notice" # Beispiel, falls der Parameter existiert
}

all_fetched_projects = []
current_last_item = None
page_count = 0
max_pages_to_fetch = 100 # Sicherheitslimit, um Endlosschleifen zu vermeiden (anpassen!)

print(f"INFO: Starte Abruf von Projektliste mit initialen Parametern: {initial_query_params}")

while True:
    page_count += 1
    print(f"INFO: Rufe Seite {page_count} ab...")
    
    current_query_params = initial_query_params.copy()
    if current_last_item:
        # WICHTIG: Ersetzen Sie 'lastItemCursor' durch den tatsächlichen Query-Parameter-Namen,
        # den die Simap-API für die Paginierung mit dem 'lastItem'-Wert erwartet!
        # Übliche Namen könnten sein: 'lastItem', 'pageCursor', 'startAfter', 'offsetToken' etc.
        # Schauen Sie in der Simap API Dokumentation (Swagger) nach!
        current_query_params["lastItem"] = current_last_item # Annahme des Parameternamens

    page_data = call_simap_api(search_url, params=current_query_params)

    if page_data and "projects" in page_data:
        projects_on_page = page_data["projects"]
        print(f"INFO: {len(projects_on_page)} Projekte auf dieser Seite gefunden.")
        all_fetched_projects.extend(projects_on_page)

        pagination_info = page_data.get("pagination")
        if pagination_info:
            print(f"INFO: Pagination Info: {pagination_info}")
            items_per_page = pagination_info.get("itemsPerPage", 20) # Standardwert falls nicht vorhanden
            # Bedingung zum Stoppen: Wenn weniger Projekte als 'itemsPerPage' geliefert wurden
            # ODER wenn 'lastItem' nicht mehr vorhanden/null ist (sicherere Methode, falls 'lastItem' immer kommt)
            if "lastItem" in pagination_info and pagination_info["lastItem"]:
                current_last_item = pagination_info["lastItem"]
                if len(projects_on_page) < items_per_page: # Wenn letzte Seite erreicht
                    print("INFO: Weniger Projekte als 'itemsPerPage' erhalten, wahrscheinlich letzte Seite.")
                    break
            else: # Kein 'lastItem' mehr in pagination oder 'lastItem' ist null
                print("INFO: Kein 'lastItem' mehr in Pagination Info oder 'lastItem' ist null. Stoppe Paginierung.")
                break
        else: # Keine Pagination-Info mehr
            print("INFO: Keine Pagination-Informationen mehr in der Antwort. Stoppe Paginierung.")
            break
        
        if page_count >= max_pages_to_fetch:
            print(f"WARNUNG: Maximal Anzahl von {max_pages_to_fetch} Seiten abgerufen. Stoppe Paginierung, um Endlosschleife zu vermeiden.")
            break
            
    else:
        print(f"FEHLER: Keine Projekte auf Seite {page_count} gefunden oder Fehler beim Abrufen. Stoppe Paginierung.")
        if page_data:
             print(f"Antwort von Simap (Seite {page_count}): {page_data}")
        break

print(f"\nINFO: Insgesamt {len(all_fetched_projects)} Projekte über {page_count} Seite(n) abgerufen.")

# Client-seitige Filterung nach pubType, falls nicht API-seitig möglich
projects_for_application = []
if all_fetched_projects:
    for project in all_fetched_projects:
        pub_type = project.get("pubType", "").lower()
        if pub_type in ["tender", "advance_notice"]:
            projects_for_application.append(project)



print(f"INFO: {len(projects_for_application)} Projekte sind 'tender' oder 'advance_notice' und werden weiter analysiert.")




--- Schritt 1: Alle relevanten Projektseiten abrufen (mit Paginierung) ---
INFO: Starte Abruf von Projektliste mit initialen Parametern: {'lang': 'de', 'processTypes': 'open', 'cpvCodes': ['72000000', '48000000'], 'newestPublicationFrom': '2025-05-27'}
INFO: Rufe Seite 1 ab...
INFO: Rufe Simap API auf: https://simap.ch/api/publications/v2/project/project-search mit Parametern: {'lang': 'de', 'processTypes': 'open', 'cpvCodes': ['72000000', '48000000'], 'newestPublicationFrom': '2025-05-27'}
INFO: 20 Projekte auf dieser Seite gefunden.
INFO: Pagination Info: {'lastItem': '20250527|16842', 'itemsPerPage': 20}
INFO: Rufe Seite 2 ab...
INFO: Rufe Simap API auf: https://simap.ch/api/publications/v2/project/project-search mit Parametern: {'lang': 'de', 'processTypes': 'open', 'cpvCodes': ['72000000', '48000000'], 'newestPublicationFrom': '2025-05-27', 'lastItem': '20250527|16842'}
INFO: 3 Projekte auf dieser Seite gefunden.
INFO: Pagination Info: {'lastItem': '20250527|17552', 'itemsPerPage'

In [57]:
import time # Für Pausen zwischen API-Aufrufen

print("\n--- Abruf der Detailinformationen für die gefilterten Projekte ---")

# Diese Liste wird die abgerufenen Detail-JSON-Objekte speichern
fetched_project_details_list = []
# Diese Liste kann Fehlerinformationen speichern, falls ein Detailabruf fehlschlägt
detail_fetch_errors = []

if 'projects_for_application' in locals() and projects_for_application:
    # Für den Testlauf können Sie die Anzahl der zu verarbeitenden Projekte begrenzen:
    # projects_to_get_details_for = projects_for_application[:2] # Z.B. nur die ersten 2
    projects_to_get_details_for = projects_for_application # Alle Projekte aus der gefilterten Liste verarbeiten
    
    print(f"INFO: Beginne mit dem Abruf von Detailinformationen für {len(projects_to_get_details_for)} Projekte.")

    for index, project_summary in enumerate(projects_to_get_details_for):
        project_id_for_detail = project_summary.get("id")
        publication_id_for_detail = project_summary.get("publicationId")
        project_title_summary = project_summary.get('title', {}).get('de', 'Unbekannter Titel')

        print(f"\n[{index + 1}/{len(projects_to_get_details_for)}] Verarbeite Projekt: '{project_title_summary}'")
        print(f"  Summary Projekt ID: {project_id_for_detail}")
        print(f"  Summary Publikations ID: {publication_id_for_detail}")

        if not project_id_for_detail or not publication_id_for_detail:
            error_message = f"FEHLER: Fehlende Projekt-ID ('{project_id_for_detail}') oder Publikations-ID ('{publication_id_for_detail}') im Summary. Überspringe Detailabruf."
            print(error_message)
            detail_fetch_errors.append({
                "summary_project_id": project_id_for_detail,
                "summary_publication_id": publication_id_for_detail,
                "summary_title": project_title_summary,
                "error": error_message
            })
            continue

        # URL für den Detail-API-Aufruf zusammenbauen
        detail_url = SIMAP_BASE_URL + SIMAP_DETAIL_ENDPOINT_TEMPLATE.format(
            projectId=project_id_for_detail, 
            publicationId=publication_id_for_detail
        )
        
        # Detail-API aufrufen
        project_details_json = call_simap_api(detail_url)

        if project_details_json:
            print(f"INFO: Details für Projekt '{project_title_summary}' (Pub-ID: {publication_id_for_detail}) erfolgreich abgerufen.")
            fetched_project_details_list.append(project_details_json)
            
            # Optional: Einen kleinen Teil der abgerufenen Details anzeigen
            # print("  Ausschnitt der Details:")
            # print(f"    Detail-Titel (DE): {project_details_json.get('project-info', {}).get('title', {}).get('de', 'N/A')}")
            # desc_snippet = project_details_json.get('procurement', {}).get('orderDescription', {}).get('de', 'N/A')
            # print(f"    Detail-Beschreibung (DE-Anfang): {desc_snippet[:100] if desc_snippet else 'N/A'}...")
        else:
            error_message = f"FEHLER: Konnte keine Details für Projekt '{project_title_summary}' (Pub-ID: {publication_id_for_detail}) abrufen."
            print(error_message)
            detail_fetch_errors.append({
                "summary_project_id": project_id_for_detail,
                "summary_publication_id": publication_id_for_detail,
                "summary_title": project_title_summary,
                "detail_url_called": detail_url,
                "error": error_message
            })
            
        # Eine höfliche Pause zwischen den API-Aufrufen einlegen
        print("INFO: Warte 1 Sekunde vor dem nächsten API-Aufruf...")
        time.sleep(1) # Warte 1 Sekunde

    print(f"\n--- Detailabruf abgeschlossen ---")
    print(f"INFO: Erfolgreich Details für {len(fetched_project_details_list)} von {len(projects_to_get_details_for)} Projekten abgerufen.")
    if detail_fetch_errors:
        print(f"WARNUNG: Bei {len(detail_fetch_errors)} Projekten gab es Fehler beim Detailabruf.")
        # print("Fehlerdetails:", json.dumps(detail_fetch_errors, indent=2, ensure_ascii=False))


    # Ausgabe der ersten paar abgerufenen Projektdetails zur Überprüfung
    if fetched_project_details_list:
        print("\n--- Erste(s) abgerufene(s) Projektdetail(s) zur Ansicht (max. 1) ---")
        # Wir geben die Struktur des ersten Elements aus, um zu sehen, ob es passt
        print(json.dumps(fetched_project_details_list[0], indent=2, ensure_ascii=False, default=str))
    else:
        print("INFO: Keine Projektdetails erfolgreich abgerufen.")

else:
    print("FEHLER: Die Liste 'projects_for_application' wurde in der vorherigen Zelle nicht erfolgreich erstellt oder ist leer.")
    print("Bitte führen Sie zuerst die Zelle zur Projektabfrage und -filterung (Zelle 4) erfolgreich aus.")

# Die Variable 'fetched_project_details_list' enthält nun die Detail-JSON-Objekte
# der Projekte, für die der Abruf erfolgreich war.
# Diese können Sie im nächsten Schritt für die OpenAI-Bewertung verwenden oder weiter analysieren.


--- Abruf der Detailinformationen für die gefilterten Projekte ---
INFO: Beginne mit dem Abruf von Detailinformationen für 87 Projekte.

[1/87] Verarbeite Projekt: 'None'
  Summary Projekt ID: 0712ca2c-7429-4f83-ac51-903b7358bc0f
  Summary Publikations ID: 4b400201-3a77-4daa-8de3-faf003175cdb
INFO: Rufe Simap API auf: https://simap.ch/api/publications/v1/project/0712ca2c-7429-4f83-ac51-903b7358bc0f/publication-details/4b400201-3a77-4daa-8de3-faf003175cdb mit Parametern: None
INFO: Details für Projekt 'None' (Pub-ID: 4b400201-3a77-4daa-8de3-faf003175cdb) erfolgreich abgerufen.
INFO: Warte 1 Sekunde vor dem nächsten API-Aufruf...

[2/87] Verarbeite Projekt: 'None'
  Summary Projekt ID: 7fafbf92-d6d7-4577-af41-0a5258d71215
  Summary Publikations ID: 43b44738-281a-4a55-90c4-c1307de76dd5
INFO: Rufe Simap API auf: https://simap.ch/api/publications/v1/project/7fafbf92-d6d7-4577-af41-0a5258d71215/publication-details/43b44738-281a-4a55-90c4-c1307de76dd5 mit Parametern: None
INFO: Details für P

In [60]:
company_profile = {
    "name": "Mesoneer GmbH",
    "domains": [
        "Identifikationssoftware",
        "Engineering (Workflow-Automation, RPA, Data Processing, Data-Plattform-Lösungen)",
        "AI & Data (Data Governance, Data Strategy, AI-Anwendungen)"
    ],
    "expertise": [
        "Process automatisation","Data Streaming","Data Plattforms",
        "Workflow Engines","IDP","RPA","Cloud","AI And GenAI",
        "Data Engineering","Data Governance and Strategy",
        "Identification Software","Kyc & Onboarding solutions"
    ],
    "technologies": ["BPMN 2.0", "Camunda BPM", "Axon Ivy", "Flowable", 
                     "Apache Kafka", "UiPath", "Microsoft Power Automate",
                     "Apache","Azure","Python","JAVA"],
    "max_contract_value_eur": 500_000
}


# 2. Function-Definition inklusive neuer Klassifikations-Felder
functions = [
    {
        "name": "enrich_project",
        "description": (
            "Analysiere ein SIMAP-Projekt, fasse es kurz zusammen, "
            "extrahiere nur deutsche Werte, ordne es einem Team zu "
            "(Products, Engineering, Data&AI), gib einen Apply-Score 1–10 in wie fern wir geignet sind, "
            "und liste fehlende Felder als MissingInfo auf."
        ),
        "parameters": {
            "type": "object",
            "properties": {
                "summary": {"type": "string"},
                "project": {
                    "type": "object",
                    "properties": {
                        "title_de": {"type": "string"},
                        "customer": {"type": "string"},
                        "location": {"type": "string"},
                        "projectNumber": {"type": "string"},
                        "projectId":{"type": "string"},
                        "publicationDate": {"type": "string"},
                        "offerDeadline": {"type": "string"},
                        "contract_start": {"type": "string"},
                        "qna_deadline":{"type": "string"},
                        "cpvCode": {
                            "type": "object",
                            "properties": {
                                "code":     {"type": "string"},
                                "label_de": {"type": "string"}
                            },
                            "required": ["code","label_de"]
                        }
                    },
                    "required": [
                        "qna_deadline,title_de","customer","location","projectId"
                        "publicationDate","offerDeadline","contract_start","cpvCode","projectNumber"
                    ]
                },
                "team": {"type": "string", "enum": ["Products","Engineering","Data&AI"]},
                "apply_score": {
                    "type": "integer",
                    "description": "1–10, in wie fern sich eine Bewerbung lohnt auf Basis unserer Expertise,Technologien und Unternehmensprofil.",
                },
                "missing_info": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "Welche relevanten Felder im JSON fehlten"
                }
            },
            "required": ["summary","project","team","apply_score","missing_info"]
        }
    }
]


# --- 2. Einzel-Projekt-Analyzer -------------------------------------------
def enrich_project(full_proj: dict, comp_profile: dict) -> dict:

    system_content = f"""
        Du bist RFP-Analyst für Mesoneer ag. Mesoneer ist Experte für:
        • Identifikationssoftware
        • Engineering (Automation, Data Processing, Plattformen)
        • AI & Data (Data Governance, Strategie, AI)

        Nutze nur die Felder aus dem SIMAP-JSON, die auf Deutsch vorliegen. Analysiere so:

        1. Schreibe eine kompakte Zusammenfassung (2–3 Sätze) in Deutsch.
        2. Extrahiere genau diese Felder aus dem JSON:
        • title_de, customer, location, publicationDate, offerDeadline, contract_start, cpvCode (code+label_de),qna_deadline,projectNumber,projectId
        3. Ordne das Projekt einem Team zu: Products, Engineering oder Data&AI.
        4. Vergib einen Apply-Score von 1–10, basierend auf:
        – Übereinstimmung mit unserem Profil (Domains, Expertise, Technologies)
        – Realistische Abschlusswahrscheinlichkeit:  
            • 1 = sehr unwahrscheinlich  
            • 10 = top-Potenzial  
        5. Liste in `missing_info` alle der sieben Felder auf, die im Input-JSON gar nicht zu finden waren.
        """
    resp = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content":
                "PROJECT_JSON =\n" + json.dumps(full_proj, ensure_ascii=False, indent=2)
                + "\n\nCOMPANY_PROFILE =\n" + json.dumps(comp_profile, ensure_ascii=False, indent=2)
            }
        ],
        functions=functions,
        function_call={"name": "enrich_project"},
        temperature=0.2
    )
    func_call = resp.choices[0].message.function_call
    return json.loads(func_call.arguments)

TARGET_KEYS = ["title_de","customer","location","publicationDate","offerDeadline","contract_start","cpvCode","qna_deadline","projectId"]

def normalize(enriched: dict) -> dict:
    proj = enriched["project"]
    for k in TARGET_KEYS:
        proj.setdefault(k, None)
    return enriched

# --- 3. Batch-Loop über alle Projekte -------------------------------------
def enrich_projects_batch(projects: list[dict], comp_profile: dict) -> list[dict]:
    enriched_all = []
    for proj in projects:
        raw = enrich_project(proj, comp_profile)   # rohes GPT-Ergebnis
        normalized = normalize(raw)                 # hier füllen wir fehlende Keys auf
        enriched_all.append(normalized)
    return enriched_all

# --- 4. Hauptprogramm: Einlesen + Speichern -------------------------------
if __name__ == "__main__":
    # a) Lese deine Projekte ein (z.B. aus Datei, DB, API)
    projects = fetched_project_details_list
    enriched_list = enrich_projects_batch(projects, company_profile)
    high_score = [p for p in enriched_list if p.get("apply_score", 0) >= 8]


    with open("enriched_projects.json", "w", encoding="utf-8") as f_out:
        json.dump(high_score, f_out, ensure_ascii=False, indent=2)

    print(f"{len(high_score)} Projekte angereichert und in 'enriched_projects.json' gespeichert.")



2 Projekte angereichert und in 'enriched_projects.json' gespeichert.


In [61]:
from datetime import datetime
def fmt_date(iso_str: str, fmt: str = "%d.%m.%Y") -> str:
    """
    Wandelt einen ISO-Datetime-String in das gewünschte Format um.
    Falls parsing fehlschlägt, wird der Original-String zurückgegeben.
    """
    try:
        # einige Deadlines kommen mit Zeit + Offset
        dt = datetime.fromisoformat(iso_str)
        return dt.strftime(fmt)
    except Exception:
        return iso_str


# -----------------------------
# 2. Slack-Block-Generator
# -----------------------------
def format_slack_blocks(proj: dict) -> list[dict]:
    team        = proj["team"]
    title       = proj["project"].get("title_de", "—")
    customer    = proj["project"].get("customer", "—")
    score       = proj.get("apply_score", 0)
    summary     = proj.get("summary", "—")
    project_number    = proj["project"].get("projectNumber", "—")
    projectId    = proj["project"].get("projectId", "—")
    offer_dl_raw    = proj["project"].get("offerDeadline", "—")
    start_raw       = proj["project"].get("contract_start", "—")
    qa_dl_raw       = proj["project"].get("qna_deadline", "—")
    cpv         = proj["project"].get("cpvCode", {})
    cpv_code    = cpv.get("code", "—")
    cpv_label   = cpv.get("label_de", "—")
    missing     = proj.get("missing_info", [])
    missing_str = ", ".join(missing) if missing else "Keine"

    offer_dl = fmt_date(offer_dl_raw, "%d.%m.%Y")
    qa_dl      = fmt_date(qa_dl_raw,    "%d.%m.%Y")
    start    = fmt_date(start_raw,  "%d.%m.%Y")

    text = (
       
        "\n"
        f":rocket: *Team: {team}*  *#{project_number}*\n"
        f"\n"
        f":file_folder: *Projekt:* {title} / {customer}\n"
        f"\n"
        f":star: *Apply Score:* *{score}*\n"
        f"\n"
        f":page_facing_up: *Zusammenfassung:*\n>{summary}\n\n"
        f":calendar:   •   *Q&A:* {qa_dl}   •   *Frist:* {offer_dl}   •   *Start:* {start} \n"
        f"\n"
        f":pushpin: *CPV:* `{cpv_code}` – {cpv_label}\n"
        "\n"
    )

    return [
        {"type": "divider"},
        {"type": "section", "text": {"type": "mrkdwn", "text": text}},
        {
            "type": "context",
            "elements": [
                {
                    "type": "mrkdwn",
                    "text": f"<https://www.simap.ch/de/project-detail/{projectId}#ausschreibung|🔗 Vollständige Ausschreibung>"
                }]
        },
        {"type": "divider"}
    ]

def post_to_slack_blocks(blocks: list[dict]):
    payload = {"blocks": blocks}
    resp = requests.post(
        webhook_url,
        json=payload,
        headers={"Content-Type": "application/json"},
        timeout=10
    )
    resp.raise_for_status()


for proj in high_score:
    blocks = format_slack_blocks(proj)
    try:
        post_to_slack_blocks(blocks)
        print(f"✅ Gesendet: {proj['project'].get('title_de')}")
    except Exception as e:
        print(f"❌ Fehler beim Senden von {proj['project'].get('title_de')}: {e}")


✅ Gesendet: Datenaustauschplattform DAP
✅ Gesendet: 3870 NUCLEO - Omnichannel-Lösung für das CVC
